# Example of the `aitlas` toolbox in the context of image segmentation

This notebook shows a sample implementation of a image segmentation using the `aitlas` toolbox.

## Import the required packages

In [ ]:
from aitlas.datasets import MassachusettsBuildingsDataset
from aitlas.models import DeepLabV3
from aitlas.utils import image_loader
from aitlas.transforms import MinMaxNormTransponse

## Visualize images and masks

In [ ]:
dataset_config = {
    "data_dir": "/media/hdd/semantic-segmentation/MassachusettsBuildings/train_splits",
    "csv_file": "/media/hdd/semantic-segmentation/MassachusettsBuildings/train.txt"
}
dataset = MassachusettsBuildingsDataset(dataset_config)

print(f"Total number of patches: {len(dataset)}")
dataset.show_image(1567);
dataset.show_image(793);

## Load train data

In [ ]:
train_dataset_config = {
    "batch_size": 4,
    "shuffle": True,
    "num_workers": 4,
    "data_dir": "/media/hdd/semantic-segmentation/MassachusettsBuildings/train_splits",
    "csv_file": "/media/hdd/semantic-segmentation/MassachusettsBuildings/train.txt",
    "transforms": ["aitlas.transforms.MinMaxNormTransponse"],
    "target_transforms": ["aitlas.transforms.Transponse"]
}
train_dataset = MassachusettsBuildingsDataset(train_dataset_config)
len(train_dataset)

## Create the model

In [ ]:
epochs = 5
model_directory = "/media/hdd/semantic-segmentation/MassachusettsBuildings/experiments"
model_config = {
    "num_classes": 2, 
    "learning_rate": 0.0001,
    "pretrained": True, 
    "threshold": 0.5,
    "metrics": ["iou"]
}

model = DeepLabV3(model_config)
model.prepare()

## Start the training

In [ ]:
model.train_model(
    train_dataset=train_dataset,
    epochs=epochs,
    model_directory=model_directory,
    run_id='1'
)

## Evalute the model using test data

In [ ]:
test_dataset_config = {
    "batch_size": 4,
    "shuffle": False,
    "num_workers": 4,
    "data_dir": "/media/hdd/semantic-segmentation/MassachusettsBuildings/test_splits",
    "csv_file": "/media/hdd/semantic-segmentation/MassachusettsBuildings/test.txt",
    "transforms": ["aitlas.transforms.MinMaxNormTransponse"],
    "target_transforms": ["aitlas.transforms.Transponse"]
}

test_dataset = MassachusettsBuildingsDataset(test_dataset_config)
len(test_dataset)

model = DeepLabV3(model_config)
model.prepare()
model_path = "/media/hdd/semantic-segmentation/MassachusettsBuildings/experiments/checkpoint.pth.tar"
model.evaluate(dataset=test_dataset, model_path=model_path)
model.running_metrics.get_scores(["iou"])

## Predictions

In [ ]:
model_path = "/media/hdd/semantic-segmentation/MassachusettsBuildings/experiments/checkpoint.pth.tar"
#labels = MassachusettsBuildingsDataset.labels
labels = ["Background", "Buildings"]
transform = MinMaxNormTransponse()
model.load_model(model_path)

image = image_loader('/media/ssd/MassachusettsBuildings/output/N-34-97-C-b-1-2_61.jpg')
fig = model.predict_masks(image, labels, transform)

image = image_loader('/media/ssd/MassachusettsBuildings/output/M-33-48-A-c-4-4_49.jpg')
fig = model.predict_masks(image, labels, transform)

image = image_loader('/media/ssd/MassachusettsBuildings/output/N-33-139-C-d-2-2_159.jpg')
fig = model.predict_masks(image, labels, transform)